In [55]:
from datenguide_python.query_execution import QueryExecutioner
import re
from functools import partial

In [73]:
class Field(object):
    internal_exec = QueryExecutioner()
    
    def __init__(self,name,return_Type,args=dict()):
        self.name = name
        self.subfields = []
        self.args = args
        self._returnType = return_Type
        
    def __str__(self):
        arg_str = str(self.args) if len(self.args) > 0 else ''
        return '\n'.join([self.name + arg_str] 
                         + [re.sub(r'(^|\n)',r'\1   ',str(sf))
                            for sf in self.subfields])
    
    def __getattr__(self,k):
        meta = self.internal_exec._get_type_info(self._returnType)
        return partial(self.add_subfield,k)
    
    def __dir__(self):
        meta = self.internal_exec._get_type_info(self._returnType)
        return (super().__dir__() + list(meta.keys()))
            
        
    @classmethod
    def toplevel(cls):
        return cls('query','Query')
    
    def info(self):
        return self.internal_exec._get_type_info(self._returnType)
        
    def add_subfield(self,field,args = {}):
        possible_fields = self.internal_exec._get_type_info(self._returnType)
        return_type = possible_fields[field].get_return_type()
        f = Field(field,return_type,args)
        self.subfields.append(f)
        return f
    

## .add_subfield style adding

In [76]:
query = Field.toplevel()
region = query.add_subfield('region',{'id':'01'})
n = region.add_subfield('name')
statistic = region.add_subfield('AENW01')
statistic2 = region.add_subfield('AENW06')
# Scalar fields don't need to be bound to variables as nothing can be done with them
y = statistic.add_subfield('year')
statistic2.add_subfield('year')
print(query)

query
   region{'id': '01'}
      name
      AENW01
         year
      AENW06
         year


## `__getattr__` style adding
This is also supported by the `__dir__` implementation of the class

In [77]:
query2 = Field.toplevel()
region2 = query2.region({'id':'01'})
n = region2.add_subfield('name') #namin conflict with attribute name of the field
statistic_2 = region2.AENW01()
statistic2_2 = region2.AENW06()
# Scalar fields don't need to be bound to variables as nothing can be done with them
y = statistic_2.year()
statistic2_2.year()
print(query2)

query
   region{'id': '01'}
      name
      AENW01
         year
      AENW06
         year
